# xView training set @JEODPP

- This notebook demonstrates how the user can have access to the **xView** collection stored at the JEODPP EOS storage system.
- More specifically, it contains different ways of retrieving the **xView** images (input images and masks containing the class labels) which can be used as training data.

**For more information:** 

- @GitLab: https://jeodpp.jrc.ec.europa.eu/apps/gitlab/jeodpp-services/training-sets-for-earth-observation-applications/-/wikis/home
- @Connected: https://connected.cnect.cec.eu.int/groups/bigdataeoss 
- @Internet: https://jeodpp.jrc.ec.europa.eu/home/

**Contacts:**  jrc-jeodpp@ec.europa.eu

**Source data:** http://xviewdataset.org/

<img src="https://cidportal.jrc.ec.europa.eu/services/shared/html/JRClogo2.png" width="200" height="200" /> <img src="https://cidportal.jrc.ec.europa.eu/services/shared/html/JRCBigDataPlatform_512.png" width="200" height="200" /> 

In [ ]:
import numpy as np
import os, fnmatch, urllib.request
import pandas as pd
import json 
import matplotlib.pyplot as plt

In [ ]:
# Please, download the scripts Query.py and gdalRead.py into the working directory
from Query import Query
from gdalRead import gdalRead

In [ ]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [ ]:
mainfolder = 'https://jeodpp.jrc.ec.europa.eu/ftp/public/MachineLearning/SatImNet'
collection = 'xView'

# Get info for the dataset

In [ ]:
# Read general info for all the datasets
df = pd.read_json(os.path.join(mainfolder, 'Table.json'))
cols = list(df.columns)
cols.remove('Feature')
df = df[['Feature']+cols]
df

In [ ]:
# Read specific info for xView
pd.set_option('display.max_colwidth', 200)
try: df.set_index('Feature', inplace=True);
except: pass
df[[collection]]

# Read the structure of the xView dataset

In [ ]:
with urllib.request.urlopen(os.path.join(mainfolder, collection, 'content_public.json')) as f:
    content = json.loads(f.read().decode())

# Get class notation

In [ ]:
classes = content['classes']
classes

# Search for images according to some criteria

In [ ]:
# Use the string 'path' as 3rd argument in case you would like to retrieve the file paths only.
# Search for tif files and class: 'Building'
query = Query(content['tree'], 
               {'genre': 'tif', 'class': ['Building']}, 'path')
query

In [ ]:
# Use the string 'path' as 3rd argument in case you would like to retrieve the file paths only.
# Search for files having specific size in terms of rows and columns
query = Query(content['tree'], 
               {'type': 'file', 'metainfo_columns': [10, 20], 'metainfo_rows': [10, 30]})
query

# Read the content of an image file

In [ ]:
# Read a mask containing class labels
infile = '/vsizip//vsicurl/https://jeodpp.jrc.ec.europa.eu/ftp/public/MachineLearning/SatImNet/xView/train/labels_tif/10.tif.zip/Building_180960.tif'
InfoMask, Mask = gdalRead(infile)
InfoMask

In [ ]:
# Read the respective image
query = Query(content['tree'], {'name': os.path.basename(os.path.dirname(infile)).replace('.zip', ''), 'class': []}, 'path')
if len(query) > 0:
    Info, Image = gdalRead(query[0])
    print(Info)

In [ ]:
# Display images
fig, axarr = plt.subplots(1, 2, figsize=(15, 10))
axarr[0].axis('off')
axarr[1].axis('off')
axarr[0].imshow(Image)
axarr[1].imshow(Mask)
plt.tight_layout(h_pad=0.1, w_pad=0.1)
plt.show()

In [ ]:
col = int(np.round((abs(Info['geotransform'][0]) - abs(InfoMask['geotransform'][0])) / Info['geotransform'][1]))
row = int(np.round((abs(InfoMask['geotransform'][3]) - abs(Info['geotransform'][3])) / Info['geotransform'][5]))

In [ ]:
# Show bounding box in white color
plt.figure(figsize=(20, 20), num=None)
tmp = Image.copy()
tmp[row:row+Mask.shape[0], col:col+Mask.shape[1], :] = 255
plt.imshow(tmp)
plt.show()
del tmp

# Imageshow

In [ ]:
# Use the string 'path' as 3rd argument in case you would like to retrieve the file paths only.
# Search for files belonging to 'train' folder and used as input images
query = Query(content['tree'], 
               {'path': 'train', 'class': []}, 'path')
query

In [ ]:
from IPython.display import display, clear_output
f, axarr = plt.subplots(2, 5)
f.set_size_inches(16, 8)
val = 0
for idx in np.arange(0,len(query),10):
    pos = 0
    for q in range(2):
        for p in np.arange(5):
            _, I = gdalRead(query[idx+pos])
            axarr[q,p].axis('off')
            axarr[q,p].imshow(I)
            pos += 1
    display(f)
    if val != 'p':
        val = input("Press Enter to continue...")
        if val == 'x':
            clear_output(wait = True)
            break
    clear_output(wait = True)

In [ ]:
# Use the string 'path' as 3rd argument in case you would like to retrieve the file paths only.
# Search for tif files and class: 'Bus'
query = Query(content['tree'], 
               {'genre': 'tif', 'class': ['Bus']}, 'path')

In [ ]:
# Press x to exit
from IPython.display import display, clear_output
f, axarr = plt.subplots(2, 5)
f.set_size_inches(10, 6)
val = 0
for idx in np.arange(0,len(query),10):
    pos = 0
    for q in range(2):
        for p in np.arange(5):
            _, I = gdalRead(query[idx+pos])
            axarr[q,p].axis('off')
            axarr[q,p].imshow(I)
            pos += 1
    display(f)
    if val != 'p':
        val = input("Press Enter to continue...")
        if val == 'x':
            clear_output(wait = True)
            break
    clear_output(wait = True)

> **In order to read many images via the _vsicurl_ driver, please use the command _gdal.VSICurlClearCache()_ after every _gdalRead_ command.**